In [1]:
%pip install -r requirements.txt

  Using cached requests-2.25.1-py2.py3-none-any.whl (61 kB)
  Using cached ultralytics-8.2.88-py3-none-any.whl (871 kB)
  Using cached chardet-4.0.0-py2.py3-none-any.whl (178 kB)
  Using cached idna-2.10-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.26.20-py2.py3-none-any.whl (144 kB)
  Using cached opencv_python-4.10.0.84-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (62.5 MB)
  Using cached torch-2.4.1-cp310-cp310-manylinux1_x86_64.whl (797.1 MB)
  Using cached torchvision-0.19.1-cp310-cp310-manylinux1_x86_64.whl (7.0 MB)
  Using cached filelock-3.15.4-py3-none-any.whl (16 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl (664.8 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-

In [2]:
%pip install mlflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 6.7 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.25.1
    Uninstalling requests-2.25.1:
      Successfully uninstalled requests-2.25.1
Note: you may need to restart the kernel to use updated packages.


In [3]:
import digitalhub as dh

# Create the project 

In [4]:
PROJECT = "demo-object-detection"
project = dh.get_or_create_project(PROJECT)

## Create the training function

In [ ]:
from digitalhub_runtime_python import handler
from digitalhub_ml.entities.utils import from_mlflow_run
import mlflow

from ultralytics import YOLO

mlflow.autolog(log_datasets=True)
with mlflow.start_run():
    # Load and train your YOLOv8 model
    model = YOLO('yolov8n.pt')
    results = model.train(data='coco8.yaml', epochs=100, imgsz=640)

    # Log parameters, metrics, and model
    mlflow.log_params({"epochs": 100, "model": "yolov8n.pt", "data": "coco8.yaml"})
    #mlflow.log_metrics({"mAP_0.5": results.map50, "mAP_0.5:0.95": results.map})
    mlflow.pytorch.log_model(model, "model")

    # End the run
    mlflow.end_run()

# utility to map mlflow run artifacts to model metadata
run_id = mlflow.last_active_run().info.run_id
model_params = from_mlflow_run(run_id)


In [9]:
%%writefile functions/train-model.py

from digitalhub_runtime_python import handler
from digitalhub_ml.entities.utils import from_mlflow_run
import mlflow

from ultralytics import YOLO

@handler()
def train(project):
    mlflow.autolog(log_datasets=True)
    with mlflow.start_run():
        # Load and train your YOLOv8 model
        model = YOLO('yolov8n.pt')
        results = model.train(data='coco8.yaml', epochs=100, imgsz=640)
    
        # Log parameters, metrics, and model
        mlflow.log_params({"epochs": 2, "model": "yolov8n.pt", "data": "coco8.yaml"})
        mlflow.pytorch.log_model(model, "model")
    
        # End the run
        mlflow.end_run()

    # utility to map mlflow run artifacts to model metadata
    run_id = mlflow.last_active_run().info.run_id
    model_params = from_mlflow_run(run_id)

    project.log_model(
        name="model-mlflow-yolo",
        kind="mlflow",
        **model_params
)

Overwriting functions/train-model.py


In [10]:
# Define the training function inside the project
train_fn = project.new_function(
                         name="train",
                         kind="python",
                         python_version="PYTHON3_9",
                         source={"source": "functions/train-model.py", "handler": "train"},
                         requirements=["ultralytics==8.2.82", "mlflow==2.15.1"])

In [ ]:
train_run = train_fn.run(action="job", local_execution=True)

# Simple training version

In [12]:

from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="coco8.yaml", epochs=100, imgsz=640)

Ultralytics YOLOv8.2.88 🚀 Python-3.10.11 torch-2.4.1+cu121 CPU (Intel Xeon Platinum 8370C 2.80GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=coco8.yaml, epochs=100, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_bo

train: Scanning /home/tenant2/ai_product_card_templates/aipc_examples/object_detection_aipc/implementation/src/datasets/coco8/labels/train.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]
val: Scanning /home/tenant2/ai_product_card_templates/aipc_examples/object_detection_aipc/implementation/src/datasets/coco8/labels/val.cache... 4 images, 0 backgrounds, 0 corrupt: 100%|██████████| 4/4 [00:00<?, ?it/s]

Plotting labels to runs/detect/train4/labels.jpg... 


optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000119, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)


2024/09/06 09:03:49 INFO mlflow.tracking.fluent: Autologging successfully enabled for statsmodels.


...

    100/100         0G     0.4633     0.4903     0.9683         13        640: 100%|██████████| 1/1 [00:00<00:00,  1.47it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  4.15it/s]

                   all          4         17      0.947      0.382      0.419       0.29



100 epochs completed in 0.039 hours.
Optimizer stripped from runs/detect/train4/weights/last.pt, 6.6MB
Optimizer stripped from runs/detect/train4/weights/best.pt, 6.5MB

Validating runs/detect/train4/weights/best.pt...
Ultralytics YOLOv8.2.88 🚀 Python-3.10.11 torch-2.4.1+cu121 CPU (Intel Xeon Platinum 8370C 2.80GHz)
Model summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  5.25it/s]


                   all          4         17      0.599      0.898      0.888      0.618
                person          3         10      0.647        0.5       0.52       0.29
                   dog          1          1      0.315          1      0.995      0.597
                 horse          1          2      0.689          1      0.995      0.598
              elephant          1          2      0.629      0.886      0.828      0.332
              umbrella          1          1      0.539          1      0.995      0.995
          potted plant          1          1      0.774          1      0.995      0.895
Speed: 0.5ms preprocess, 41.5ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs/detect/train4
MLflow: results logged to runs/mlflow
MLflow: disable with 'yolo settings mlflow=False'


In [14]:
results = model(["datasets/coco8/images/val/000000000061.jpg"])


0: 512x640 1 person, 1 elephant, 71.8ms
Speed: 2.0ms preprocess, 71.8ms inference, 0.8ms postprocess per image at shape (1, 3, 512, 640)


In [15]:
# Process results list
for result in results:
    boxes = result.boxes  # Boxes object for bounding box outputs
    masks = result.masks  # Masks object for segmentation masks outputs
    keypoints = result.keypoints  # Keypoints object for pose outputs
    probs = result.probs  # Probs object for classification outputs
    obb = result.obb  # Oriented boxes object for OBB outputs
    result.show()  # display to screen
    result.save(filename="result.jpg")  # save to disk

/usr/bin/xdg-open: 882: www-browser: not found
/usr/bin/xdg-open: 882: links2: not found
/usr/bin/xdg-open: 882: elinks: not found
/usr/bin/xdg-open: 882: links: not found
/usr/bin/xdg-open: 882: lynx: not found
/usr/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpf6krevhd.PNG'
